In [9]:
import tqdm
import pandas as pd
from lxml import etree
import resource

In [10]:
XML_FILE = "xml_data/sample.xml"
CSV_OUTPUT = "csv_data/sample-output.csv"

In [11]:
def strip_tag_name(t):
    t = t.tag
    idx = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [12]:
class PageParser(object):
    def __init__(self):
        self.isPage = False
        self.pages = []
        self.current_page = {}
        
    
    def handle(self, event, element):
        stipped_elem = strip_tag_name(element)
        if event == "start" and stipped_elem == "page":
            self.isPage = True
        elif event == "end" and stipped_elem == "page":
            self.isPage = False
            self.finalize_page()
        
        if event == "end" and self.isPage:
            self.parse_page_element(element)
            
    def finalize_page(self):
        self.pages.append(self.current_page)
        self.current_page = {}
    
    def parse_page_element(self, element):
        stripped_tag = strip_tag_name(element)
        if element.text:
            texts = [element.text.strip()] + [child.tail.strip() for child in element if child.tail]
            self.current_page[stripped_tag] = " ".join(texts)
        

In [13]:
page_parser = PageParser()

In [14]:
for event, elem in tqdm.tqdm_notebook(etree.iterparse(XML_FILE, events=('start', 'end'))):
    page_parser.handle(event, elem)

In [15]:
df = pd.DataFrame(page_parser.pages)

In [16]:
df.to_csv(CSV_OUTPUT)